# Analyzing Quantum Volume Errors
This notebook analyzes the error rates required for achieving Quantum Volume at a particular depth. For a given m = depth = number of qubits, plot the HOG for np.logspace outputs to view when it crosses the 2/3rds probability threshold.

In [ ]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")

In [ ]:
import cirq
import cirq_google

# Configuration parameters. Feel free to mess with these!
num_circuits = 10
depth = 4
num_samplers = 50
repetitions = 10_000
device = cirq_google.Sycamore

print(f"Configuration: depth {depth} with " f"{num_circuits} circuits of {num_samplers} samplers")

In [ ]:
# Run the Quantum Volume algorithm over the above parameters.

import numpy as np
from cirq.contrib import quantum_volume, routing

errors = np.logspace(-1, -4, num=num_samplers)
samplers = [
    cirq.DensityMatrixSimulator(
        noise=cirq.ConstantQubitNoiseModel(qubit_noise_gate=cirq.DepolarizingChannel(p=error))
    )
    for error in errors
]


def optimize(circuit):
    return cirq.optimize_for_target_gateset(circuit, gateset=cirq.SqrtIswapTargetGateset())


result = quantum_volume.calculate_quantum_volume(
    num_circuits=num_circuits,
    depth=depth,
    num_qubits=depth,
    device_graph=routing.gridqubits_to_graph_device(device.metadata.qubit_set),
    samplers=samplers,
    compiler=optimize,
    repetitions=repetitions,
)

In [ ]:
# Create a chart that plots the HOG rate relative to the simulated error ratio.

from matplotlib import pyplot as plt
import statistics


def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i : i + n]


split = chunks([res.sampler_result for res in result], num_circuits)
fig, axs = plt.subplots()
axs.plot(errors, [statistics.mean(chunk) for chunk in split])

# Line markers for asymptotic ideal heavy output probability and the ideal Heavy
# Output Generation threshold.
axs.axhline((1 + np.log(2)) / 2, color='tab:green', label='Asymptotic ideal', linestyle='dashed')
axs.axhline(2 / 3, label='HOG threshold', color='k', linestyle='dotted')
plt.xscale('log')
axs.set_ybound(0.4, 1)
axs.set_xlabel("error rate")
axs.set_ylabel("est. heavy output probability")
fig.suptitle(f'HOG probability by simulated error rate for d={depth}')